2022/5/3<br>

This ocde is for getting retardation, azimuthal angle and MFA images based on PolSope algorithm.<br>
After conversion, all images are concatenated by Phase-only-orrelated method.<br>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import cv2
import itertools
import os, sys
import gc
import psutil
from tqdm import tqdm
import skimage
from skimage.morphology import disk
from skimage.filters import threshold_otsu, threshold_triangle, rank
from skimage.util import img_as_ubyte
import common.Ret_MFA_func
import multiprocessing as mp
from multiprocessing import Pool
from common.Stitcher import Stitcher
import copy

In [2]:
#MFA coversion
#parameters of optical anistropy of cellulose
no=1.529 #refractive index of ordinary light
ne=1.599 #refractive index of extraordinary light
cellulose_ratio=0.5

#
core_num=3
bg_file_num=4

In [3]:
current_path=os.getcwd()
image_path=os.path.join(current_path, "POM")
species=os.listdir(image_path)
npz_path=os.path.join(current_path, "npz_file")
save_path=os.path.join(current_path, "fig_save")

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

#find samlpe name from dierctory
sample_list=os.listdir(image_path)
thickness_list=[i.split("_")[-1] for i in sample_list]

In [4]:
file_path_list=[]
sample_name_list=[]
sample_position_list=[]

for sample_name in tqdm(sample_list):
    sample_path=os.path.join(image_path, str(sample_name))
    
    
    file_path_list_temp=[]
    sample_position_list_temp=[]
    
    for root, dirs, files in os.walk(os.path.join(image_path)):
        
        if (len(files)>0):
            file_path_extract=[os.path.join(root, files[i]) for i in range(len(files)) if sample_name in root]
            file_path_list_temp.append(file_path_extract)
        else:
            pass
    
    file_path_list_temp=list(itertools.chain.from_iterable(file_path_list_temp))
    sample_position_list_temp=[file_path_list_temp[3*i].split("\\")[-2].split("_") 
                                   for i in range((len(file_path_list_temp)-bg_file_num)//3)]
    
    file_path_list.append(file_path_list_temp)
    sample_position_list.append(sample_position_list_temp)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 105.29it/s]


**Convert a pair of three images to a retardation, azimuthal angle and MFA image.**

In [5]:
#call class
polscope_func=Ret_MFA_func.Polscope_func
core_num=3


#ret, azimuthal angle & MFA conversion
#all conversion processes are done folder by folder.

for i in range(len(file_path_list)):
    #set target list
    target_list=file_path_list[i]
    im_list=[]
    
    for target in target_list:

        #load image files
        im=cv2.imread(target, -1)
        im=im/65535
        im_height, im_width=im.shape
        im=im.flatten()
        im_list.append(im)


    #set main and bg_im
    #bg_im and bg_setting_im
    im_bg_list=im_list[-1*(bg_file_num):]
    im_bg=im_bg_list[0]
    im_bg_setting=im_bg_list[1:]

    #main image
    im_main_list=im_list[:-1*(bg_file_num)]

    #set list
    ret_im_list_temp=[]
    ret_binary_im_list_temp=[]
    angle_im_list_temp=[]
    MFA_im_list_temp=[]


    #polscope conversion & MFA conversion
    for j in range(len(im_main_list)//3):
        im_setting=im_main_list[j*3:(j+1)*3]

        #define term A, B
        A=polscope_func.term_A_three(im_setting[0], im_setting[1], im_setting[2], im_bg)
        B=polscope_func.term_B_three(im_setting[0], im_setting[1], im_setting[2], im_bg)

        A_bg=polscope_func.term_A_three(im_bg_setting[0], im_bg_setting[1], im_bg_setting[2], im_bg)
        B_bg=polscope_func.term_B_three(im_bg_setting[0], im_bg_setting[1], im_bg_setting[2], im_bg)

        A_true=A-A_bg
        B_true=B-B_bg

        delta_array=[]
        phi_array=[]

        #multiprocessing
        #call class
        polscope_mp=Ret_MFA_func.Polscope_mp(core_num=core_num, term_A=A_true, term_B=B_true)


        ######################
        # Polscope converion #
        ######################

        #set core numbers
        with Pool(core_num) as p:
            #calculation MFA using cores 
            callback_ret = p.map(polscope_mp.calc_delta_three_mp, range(core_num))
            callback_phi = p.map(polscope_mp.calc_phi_three_mp, range(core_num))

        #combine results
        delta_array=[]
        phi_array=[]
        for k in range(len(callback_ret)):
            delta_array.extend(callback_ret[k])
            phi_array.extend(callback_phi[k])

        delta_array=np.asarray(delta_array)
        phi_array=np.asarray(phi_array)

        #replace nan to mean value
        delta_array=np.nan_to_num(delta_array, nan=np.mean(delta_array), copy=False)
        phi_array=np.nan_to_num(phi_array, nan=np.mean(phi_array), copy=False)
        ret_array=np.asarray(delta_array)*180/np.pi*546/360
        angle_array=np.asarray(phi_array)*180/np.pi

        #MFA convnversion
        d=float(thickness_list[i])*cellulose_ratio*1000

        #call class
        MFA_func=Ret_MFA_func.MFA_func_mp(core_num=core_num, target_array=ret_array, d=d)

        #################
        # MFA converion #
        #################

        #set core numbers
        with Pool(core_num) as p:
            #calculation MFA using cores 
            callback_MFA = p.map(MFA_func.MFA_loop_polscope, range(core_num))

        #combine results
        MFA_array=[]
        for k in range(len(callback_MFA)):
            MFA_array.extend(callback_MFA[k])

        MFA_array=np.asarray(MFA_array)

        #reshape all conversion results
        ret_mat=ret_array.reshape((im_height, im_width))
        angle_mat=angle_array.reshape((im_height, im_width))
        MFA_mat=MFA_array.reshape((im_height, im_width))

        #save result
        ret_im_list_temp.append(ret_mat)
        angle_im_list_temp.append(angle_mat)
        MFA_im_list_temp.append(MFA_mat)

        #binarization
        im=np.nan_to_num(ret_mat)
        threshold = threshold_triangle(im)
        img=np.where(im<threshold, 255, 0)
        ret_binary_im_list_temp.append(img)          


    #save them as npz
    #set path
    npz_temp_save_path=os.path.join(npz_path, "conversion_results", sample_list[i])

    if os.path.exists(npz_temp_save_path)==False:
        os.makedirs(npz_temp_save_path)

    np.savez_compressed(os.path.join(npz_temp_save_path, str(sample_list[i])+".npz"), 
                       ret_im_list=ret_im_list_temp, angle_im_list=angle_im_list_temp, MFA_im_list=MFA_im_list_temp, 
                       ret_binary_im_list=ret_binary_im_list_temp, sample_position=sample_position_list[i], 
                       sample_info=sample_list[i])

In [6]:
#delete list to release memory
del ret_im_list_temp, ret_binary_im_list_temp, angle_im_list_temp, MFA_im_list_temp
del angle_array, delta_array, phi_array, ret_array, MFA_array
del ret_mat, angle_mat, MFA_mat, img
gc.collect()

24

**Reload image files and stitch them**

In [5]:
#call class
stitcher=Stitcher()

#load npz file and all data sample by sample
for root, dirs, files in os.walk(os.path.join(npz_path, "conversion_results")):
    if len(files)>0:
        for file in files:
            npz_file_path=os.path.join(root, file)
            npz_file=np.load(npz_file_path, allow_pickle=True)
            
            ret_im_list=npz_file["ret_im_list"]
            ret_binary_im_list=npz_file["ret_binary_im_list"]
            angle_im_list=npz_file["angle_im_list"]
            MFA_im_list=npz_file["MFA_im_list"]
            sample_position=npz_file["sample_position"]
            sample_info=npz_file["sample_info"]
            
            #image registration is done by two steps
            #1st step: images are stitched in the horizontal direction (result image has oblong shape)
            #2nd step: images generated in 1st step are sticthed in the longitudinal direction to make one large image.  
            
            
            #image registration
            height_div=int(sample_position[-1][0])+1 #number of rows
            
            #set list
            binary_im_stitch_list=[]
            ret_im_stitch_list=[]
            angle_im_stitch_list=[]
            MFA_im_stitch_list=[]
            
            ############
            # 1st step #
            ############
            
            for i in tqdm(range(height_div)):
                #set target image. process is done row by row
                target_im_binary_list=list(ret_binary_im_list[i*len(ret_binary_im_list)//height_div : 
                                                              (i+1)*len(ret_binary_im_list)//height_div])
                target_im_ret_list=list(ret_im_list[i*len(ret_im_list)//height_div : (i+1)*len(ret_im_list)//height_div])
                target_im_angle_list=list(angle_im_list[i*len(angle_im_list)//height_div : (i+1)*len(angle_im_list)//height_div])
                target_im_MFA_list=list(MFA_im_list[i*len(MFA_im_list)//height_div : (i+1)*len(MFA_im_list)//height_div])
                
                #if only one image exists in each row, first step is skipped.
                if len(target_im_binary_list)<2:
                    binary_im_stitch_list.append(target_im_binary_list[0])
                    ret_im_stitch_list.append(target_im_ret_list[0])
                    angle_im_stitch_list.append(target_im_angle_list[0])
                    MFA_im_stitch_list.append(target_im_MFA_list[0])
                    continue

                #set count
                n=0
                while(True):
                    if n==0:
                        im_num=len(target_im_binary_list)

                        #set list for temporary result
                        binary_image_result=[]
                        ret_image_result=[]
                        angle_image_result=[]
                        MFA_image_result=[]

                        while(True):
                            #first image is poped put from list
                            binary_image_1=target_im_binary_list.pop(0)
                            binary_image_2=target_im_binary_list[0]
                            ret_image_1=target_im_ret_list.pop(0)
                            ret_image_2=target_im_ret_list[0]
                            angle_image_1=target_im_angle_list.pop(0)
                            angle_image_2=target_im_angle_list[0]
                            MFA_image_1=target_im_MFA_list.pop(0)
                            MFA_image_2=target_im_MFA_list[0]

                            #set image pair
                            binary_images=stitcher.image_dimension_adj(binary_image_1, binary_image_2, padding=255, 
                                                                       criterion="A")
                            ret_images=stitcher.image_dimension_adj(ret_image_1, ret_image_2, 
                                                                    padding=0, criterion="A")
                            angle_images=stitcher.image_dimension_adj(angle_image_1, angle_image_2, 
                                                                      padding=0, criterion="A")
                            MFA_images=stitcher.image_dimension_adj(MFA_image_1, MFA_image_2, 
                                                                      padding=0, criterion="A")

                           
                            #sttching by POC
                            binary_image_stitch_temp, ret_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, ret_images)
                            _, angle_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, angle_images)
                            _, MFA_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, MFA_images)

                            binary_image_result.append(binary_image_stitch_temp)
                            ret_image_result.append(ret_image_stitch_temp)
                            angle_image_result.append(angle_image_stitch_temp)
                            MFA_image_result.append(MFA_image_stitch_temp)

                            #if length of original image list becomes 1, break loop 
                            list_len=len(target_im_binary_list)

                            if list_len==1:
                                break

                        n+=1
                        result_list_len=len(binary_image_result)

                        if result_list_len==1:
                            break

                    if n>0:
                        #set count
                        m=0

                        while(True):
                            #select target images to be stictched
                            #first image is poped put from list
                            binary_image_1=binary_image_result.pop(0)
                            binary_image_2=binary_image_result[0]
                            ret_image_1=ret_image_result.pop(0)
                            ret_image_2=ret_image_result[0]
                            angle_image_1=angle_image_result.pop(0)
                            angle_image_2=angle_image_result[0]
                            MFA_image_1=MFA_image_result.pop(0)
                            MFA_image_2=MFA_image_result[0]

                            #set image pair
                            binary_images=stitcher.image_dimension_adj(binary_image_1, binary_image_2, padding=255, 
                                                                       criterion="A")
                            ret_images=stitcher.image_dimension_adj(ret_image_1, ret_image_2, 
                                                                    padding=0, criterion="A")
                            angle_images=stitcher.image_dimension_adj(angle_image_1, angle_image_2, 
                                                                      padding=0, criterion="A")
                            MFA_images=stitcher.image_dimension_adj(MFA_image_1, MFA_image_2, 
                                                                      padding=0, criterion="A")


                            #sttching by POC
                            binary_image_stitch_temp, ret_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, ret_images)
                            _, angle_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, angle_images)
                            _, MFA_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, MFA_images)

                            #obtained results are pushed into the list
                            binary_image_result.append(binary_image_stitch_temp)
                            ret_image_result.append(ret_image_stitch_temp)
                            angle_image_result.append(angle_image_stitch_temp)
                            MFA_image_result.append(MFA_image_stitch_temp)

                            m+=1
                            if m==(result_list_len-1):
                                binary_image_result=binary_image_result[1:]
                                ret_image_result=ret_image_result[1:]
                                angle_image_result=angle_image_result[1:]
                                MFA_image_result=MFA_image_result[1:]
                                break

                        n+=1
                        result_list_len=len(binary_image_result)

                        if result_list_len==1:
                            break

                binary_im_stitch_list.append(binary_image_result[0])
                ret_im_stitch_list.append(ret_image_result[0])
                angle_im_stitch_list.append(angle_image_result[0])
                MFA_im_stitch_list.append(MFA_image_result[0])
            
            
            ############
            # 2nd step #
            ############
            
            #Finally, image registration of upper and bottom part is conducted
            i=0
            while(True):
                if i==0:
                    im_num=len(binary_im_stitch_list)

                    #in the first time, empty list is set for saving temporary stitching images
                    binary_image_result=[]
                    ret_image_result=[]
                    angle_image_result=[]
                    MFA_image_result=[]

                    j=0
                    while(True):
                        #select target images to be stictched
                        #first image is poped put from list
                        binary_image_1=binary_im_stitch_list.pop(0)
                        binary_image_2=binary_im_stitch_list[0]
                        ret_image_1=ret_im_stitch_list.pop(0)
                        ret_image_2=ret_im_stitch_list[0]
                        angle_image_1=angle_im_stitch_list.pop(0)
                        angle_image_2=angle_im_stitch_list[0]
                        MFA_image_1=MFA_im_stitch_list.pop(0)
                        MFA_image_2=MFA_im_stitch_list[0]

                        #adjust dimension of images
                        binary_images=stitcher.image_dimension_adj(binary_image_1, binary_image_2, padding=255, 
                                                                   criterion="A")
                        ret_images=stitcher.image_dimension_adj(ret_image_1, ret_image_2, 
                                                            padding=0, criterion="A")
                        angle_images=stitcher.image_dimension_adj(angle_image_1, angle_image_2, 
                                                                  padding=0, criterion="A")
                        MFA_images=stitcher.image_dimension_adj(MFA_image_1, MFA_image_2, 
                                                                  padding=0, criterion="A")

                        #sttching by POC
                        binary_image_stitch_temp, ret_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, ret_images)
                        _, angle_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, angle_images)
                        _, MFA_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, MFA_images)

                        binary_image_result.append(binary_image_stitch_temp)
                        ret_image_result.append(ret_image_stitch_temp)
                        angle_image_result.append(angle_image_stitch_temp)
                        MFA_image_result.append(MFA_image_stitch_temp)
                        
                        # break condition
                        j+=1
                        list_len=len(binary_im_stitch_list)

                        if list_len==1:
                            break
                    
                    #break condition
                    i+=1
                    result_list_len=len(binary_image_result)

                    if result_list_len==1:
                        break


                if i>0:
                    j=0

                    while(True):
                        #select target images to be stictched
                        #first image is poped put from list
                        binary_image_1=binary_image_result.pop(0)
                        binary_image_2=binary_image_result[0]
                        ret_image_1=ret_image_result.pop(0)
                        ret_image_2=ret_image_result[0]
                        angle_image_1=angle_image_result.pop(0)
                        angle_image_2=angle_image_result[0]
                        MFA_image_1=MFA_image_result.pop(0)
                        MFA_image_2=MFA_image_result[0]

                        #adjust dimension of images
                        binary_images=stitcher.image_dimension_adj(binary_image_1, binary_image_2, padding=255, 
                                                                   criterion="A")
                        ret_images=stitcher.image_dimension_adj(ret_image_1, ret_image_2, 
                                                                padding=0, criterion="A")
                        angle_images=stitcher.image_dimension_adj(angle_image_1, angle_image_2, 
                                                                  padding=0, criterion="A")
                        MFA_images=stitcher.image_dimension_adj(MFA_image_1, MFA_image_2, 
                                                                  padding=0, criterion="A")

                        #sttching by POC
                        binary_image_stitch_temp, ret_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, ret_images)
                        _, angle_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, angle_images)
                        _, MFA_image_stitch_temp=stitcher.POC_reference_criterion(binary_images, MFA_images)

                        #obtained results are pushed into the list
                        binary_image_result.append(binary_image_stitch_temp)
                        ret_image_result.append(ret_image_stitch_temp)
                        angle_image_result.append(angle_image_stitch_temp)
                        MFA_image_result.append(MFA_image_stitch_temp)

                        j+=1
                        if j==(result_list_len-1):
                            binary_image_result=binary_image_result[1:]
                            ret_image_result=ret_image_result[1:]
                            angle_image_result=angle_image_result[1:]
                            MFA_image_result=MFA_image_result[1:]
                            break

                    i+=1
                    result_list_len=len(binary_image_result)

                    if result_list_len==1:
                        break
            
            #set path
            stitch_im_save_path=os.path.join(save_path, "stitching_results")
            if os.path.exists(stitch_im_save_path)==False:
                os.makedirs(stitch_im_save_path)
            
            #save result as image
            fig, ax=plt.subplots(1, 3, figsize=(17, 4))
            im1=ax[0].imshow(ret_image_result[0], cmap="gist_ncar")
            pp=fig.colorbar(im1, ax=ax[0])
            pp.mappable.set_clim(0, 137)

            im2=ax[1].imshow(angle_image_result[0], cmap="jet")
            pp=fig.colorbar(im2, ax=ax[1])
            pp.mappable.set_clim(0, 180)

            im3=ax[2].imshow(MFA_image_result[0], cmap="gist_ncar")
            pp=fig.colorbar(im3, ax=ax[2])
            pp.mappable.set_clim(0, 60)
            plt.subplots_adjust(wspace=0.1)
            plt.tight_layout()
            plt.savefig(os.path.join(stitch_im_save_path, str(os.path.splitext(os.path.basename(npz_file_path))[0])+".png"), dpi=450)
            plt.close()
            
            #save result as npz
            stitch_save_path=os.path.join(npz_path, "stitching_results")
            
            if os.path.exists(stitch_save_path)==False:
                os.makedirs(stitch_save_path)
                
            np.savez_compressed(os.path.join(stitch_save_path, str(sample_info)+".npz"),
                               ret_binary_im=binary_image_result[0], ret_im=ret_image_result[0], 
                               angle_im=angle_image_result[0], MFA_im=MFA_image_result[0], 
                               sample_info=sample_info)
            

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.94s/it]
